<a href="https://colab.research.google.com/github/chadmh/Short-Hands-on-Tutorial-for-Deep-Learning-in-Tensorflow/blob/master/2_Background_Theory_to_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2.1 Linear Regression
One of the best introductions I have seen to the concept of deep learning begins with simple linear regression.  In its most basic form, linear regression attempts to estimate the slope (w) and intercept (b) of a line given some set of noisy data.

> (2.1) *y = w * x + b*

In multiple dimensions, *w* becomes an M x N matrix and *b* becomes a vector.  In the context of machine learning, *x* is commonly called the predictors or features and *y* is the output or labels.

One of the features of linear algebra is that the product of two matrices *w1* and *w2* is simply a third matrix.  This allows linear regression problems to train very quickly, but also severely limits them to only solving linear problems.  In other words repeated application of the linear equation is simply another linear equation.

# 2.2 Deep Learning

Deep learning takes the equation 1.1 and wraps it in a nonlinear function, *f()*, called the activation function.

> (2.2) *y=f(w * x + b)*

This addition means that repeated application of equation 2.2 no longer reduces to the same form of equation.  Instead, repeatedly applying equation 2.2 takes on the properties of a universal function approximator allowing the model to be trained on a wide variety of problems.  The resulting equation can be written as follows:

> (2.3) *y = LN(...L2(L1(x))...)*

where *Ln* is defined as *fn(wn * xn + bn)*.  Typically, the *Ln* are referred to as the layers of the neural network.  The first layer is commonly called the input layer; the last, the output layer; and the rest, the hidden layers. The network is considered "deep" if the number of hidden layers, N, is more than 1.  Modern deep neural networks (DNNs) may contain hundreds of layers.  

### 2.2.1 Training

Training is the process of using input data, *x*, and labels, *y*, to determine the values of the *wn* matrices and *bn* vectors that minimize the error between the predicted *y* and the truth *y*.  This is done using an optimizer.

The conceptually simplest optimizer is gradient descent.  Gradient descent finds the derivative of the error between the predicted and truth *y*s with repect to each dimension of *xn*.  The chain rule of calculus is used to break each layer out separately which keeps the calculations simple, though tedious.  The *wn* and *bn* weights are then updated to move the system down the gradient towards a minima.  Training is considered finished when the minima is reached.

While gradient descent is conceptually simple, it is also numerically inefficient.  Newer optimizers build on this concept by implementing techniques to avoid local minima, improve numerical stability, and more effectively manage computational resources. Tensorflow implements a [number of optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) better suited to the need of DNNs.  One of the most popular is Adam.

### 2.2.2 Activation Functions

The choice of activation function plays a significant role in deep learning.  Common choices are the rectified linear unit (ReLU), sigmoid, and hypertangent (tanh) functions.

The [sigmoid function](https://en.wikipedia.org/wiki/Sigmoid_function) is an S-shaped curve that has the nice property of collapsing an infinite range onto the range [0, 1].  It is still commonly used for the final layer of the network when a probability value is needed.  It is rarely used for the hidden layers of the DNN due to what is called the vanishing gradient problem.  In a nutshell, because the sigmoid function is nearly flat when the input is far from zero, it creates a couple of problems when repeatedly used:
*   Repeatedly differentiating the sigmoid for each layer in the network can cause the overall gradient to be nearly as small as the floating point error of the computer leading to inaccurate gradient evaluations.
*    An optimizer generally take smaller steps when the gradient gets smaller to avoid overshooting the minimum error point.  The flat regions of the sigmoid function can confuse the optimizer leading to extremely inefficient optimizations.

The hyperbolic tangent or [tanh function](https://mathworld.wolfram.com/HyperbolicTangent.html) can be thought of as a shifted and rescaled version of the sigmoid that collapses an infinite range onto [-1, 1].  Like the sigmoid, it is only used for output layers and is useful when symmetry around zero is needed.

The Rectified Linear Unit or [ReLU function](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) is simply *y = max(0, x)*.  It has the advantage of a very fast derivative calculation and a lack of problems with vanishing gradients (assuming the initial weights are not all zero or negative). It is one of the most common choices for the hidden layers of a neural network.

Tensorflow supports numerous [additional activation functions](https://www.tensorflow.org/api_docs/python/tf/keras/activations).  Many of these are variations of ReLU, while others are used for specific classes of networks.




### 2.2.3 Regularization

Any machine learning problem has to deal with the trade-off between bias and variance.  Bias occurs when the model used for training is not complex enough to represent the patterns in the data.  Variance occurs when the model is too complex and overfits the data, or begins to capture random patterns in the data's noise.

Regularization encourages a complex model to self-simplify, so that the model designer doesn't have to endlessly fiddle with the network architecture.  The basic idea is to apply some sort of penalty to model complexity that encourages the model either set as many weights to zero as possible while still capturing essential details or to use its complexity to build in redundancy.

One method of regularization is to set some form of penalty term based on the number of non-zero weights and to add this penalty to the error that the optimizer is trying to minimize.  Another method, called drop out, forces random weights in a given layer to zero each training iteration in order to push the model to build in redundancy.

Another form of regularization, called [batch normalization](https://arxiv.org/abs/1502.03167), forces a layer's output tensor to have Gaussian-distributed values (mean:0, standard deviation:1).  This approach has become extremely popular in model architecture because it not only provides powerful regularization, but also reduces sensititvity to initial weight values and allows faster learning rates which reduces training time.  Batch normalization reduces what the technique's authors call *internal covariant shift*, which loosely means that the tensor statistical properties remain consistent and near the origin (where our activation function is most non-linear) from layer to layer.


### 2.2.4 Loss Functions

A loss function is a function that computes the error between the truth labels of a dataset and the predicted labels computed by a model.  

#### 2.2.4.1 Common Loss Functions for Regression
Consider this example: The truth value of a data sample is 2 while the model predicted 2.7.  The error, or loss, in this instance is could be computed as 2.7 - 2 = 0.7.  Since we don't want negative losses to cancel out positive (and make the model look artificially good), we might take the absolute value before averaging (This is called L1 loss.), or we could square, average, and take the square root of the average (This is the RMS or L2 Loss.).  Both of these are common in regression problems.

#### 2.2.4.2 Common Loss Function for Classification
Consider this example: A data sample in a binary classification problem has a truth label of [0, 1] (it belongs to class 2, not class 1).  The model prediction is [0.2, 0.8].  One method to compute a loss function is to use the dot product of the prediction and truth vectors.  In the example above this gives

> *dot([0, 1], [0.2, 0.8]) = 0 * 0.2 + 1 * 0.8 = 0.8*

Because probabilities are positive, both vectors are confined to the first quadrant, and the result will always be positive.  As the vectors approach each other, the dot product gets closer to one.  As they become more orthogonal, the dot product gets closer to zero.  To convert this to a loss function, one simply needs to subtract the result from one and average over all samples.

In fact, the dot product approach is very similar to a common loss function called Categorical Crossentropy.  The main difference is that Categorial Crossentropy takes the negative log of the prediction vector rather than subtracting the result from one.  It also sums rather than averages to reduce computation.

The best loss function depends on the type of problem at hand.  While the above loss functions are sufficient for many cases, you may find times when you need to adapt or customize the loss function to capture specific features of your data set in your model.  One such example might occur when different classes have different levels of importance in the classification problem.